### Integrate

Test whether the Beckmann microfacet distribution is a true PDF (integrates to 1).

Define a function that integrates a PDF over solid angles. We assume the PDF is rotationally symmetric and does not depennd of $\phi$. First, integrate over $\theta$ from 0 to 90 degrees. Then, multiply the result by $2 \pi$ to include the integral over $\phi$.

In [1]:
import math
import vec3

def integrate(PDF):
    di = math.radians(1)
    pdf_sum = 0
    for i in range(0,91):
        ri = math.radians(i)
        H = vec3.FromAngle(ri)
        pdf_sum += PDF(H) * math.sin(ri) * di # sini because we are integrating dw
    return 2 * math.pi * pdf_sum

In [2]:
import microfacet

alpha = .1
beckmann = microfacet.Beckmann(alpha, alpha)
print('Beckmann(alpha={}) = {}'.format(alpha, integrate(beckmann.Pdf)))

Beckmann(alpha=0.1) = 0.9949076750901112


This shows that the Beckmann microfacet distribution is properly normalized.

Implement the integration by integrating over $d \cos \theta$. 

In [3]:
def integrate_dcos(PDF):
    N = 100000
    dcosi = 1/N
    pdf_sum = 0.0
    for i in range(N+1):
        cosi = i/N
        H = vec3.FromCosAngle(cosi)
        pdf_sum += PDF(H) * dcosi
    return 2 * math.pi * pdf_sum

In [4]:
print('Beckmann(alpha={}) = {}'.format(alpha, integrate_dcos(beckmann.Pdf)))

Beckmann(alpha=0.1) = 1.0010003283333149


This also yields 1, again showing the Beckmann distribution is properly normalized.

### Consistency

Check that the average projected area is equal to $\cos \theta_i$.

In [5]:
import vgroove

EPSILON = 1e-2

def check(beckmann, a):
    a = math.radians(a)
    I = vec3.FromAngle(a)
    N = vec3.FromAngle(0)
    n = 100000
    dcos = 1/n
    dcos_sum= 0.0
    for i in range(n+1):
        cosh = i/n
        H = vec3.FromCosAngle(cosh)
        G = vgroove.G1(N,H,I)
        HdotI = vec3.dot(H,I)
        dcos_sum += 0.5 * G * beckmann.D(H) * HdotI * dcos
    return 2 * math.pi * dcos_sum

In [6]:
theta = 30
costheta1 = check(beckmann, theta)
print('cos({}) = {}'.format(theta, costheta1))
costheta2 = math.cos(math.radians(theta))
print('cos({}) = {}'.format(theta, costheta2))
assert abs(costheta1 - costheta2) < EPSILON

cos(30) = 0.8668917135332129
cos(30) = 0.8660254037844387


In [7]:
theta = 0
costheta1 = beckmann.visible(math.radians(theta))
print('cos({}) = {}'.format(theta, costheta1))
costheta2 = math.cos(math.radians(theta))
print('cos({}) = {}'.format(theta, costheta2))
assert abs(costheta1 - costheta2) < EPSILON

cos(0) = 1.0010003283333149
cos(0) = 1.0


In [8]:
theta = 30
costheta1 = beckmann.visible(math.radians(theta))
print('cos({}) = {}'.format(theta, costheta1))
costheta2 = math.cos(math.radians(theta))
print('cos({}) = {}'.format(theta, costheta2))
assert abs(costheta1 - costheta2) < EPSILON

cos(30) = 0.8668917135332129
cos(30) = 0.8660254037844387
